In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

import sqlite3

In [2]:
df = pd.read_csv('/home/lucas/repos/coronapolation/prediction/Influenza/All_raw_test1.csv', encoding='utf-16')
df = df.fillna(0)
df

,date/Kreis,LK Breisgau-Hochschwarzwald,LK Emmendingen,SK Freiburg i.Breisgau,LK Konstanz,LK Lörrach,LK Ortenaukreis,LK Rottweil,LK Schwarzwald-Baar-Kreis,LK Tuttlingen,...,LK Saalfeld-Rudolstadt,LK Schmalkalden-Meiningen,LK Sömmerda,LK Sonneberg,SK Suhl,LK Unstrut-Hainich-Kreis,LK Wartburgkreis,SK Weimar,LK Weimarer Land,Unbekannt
0,2001-KW01,11.0,0.0,3.0,3.0,2.0,6.0,1.0,4.0,4.0,...,7.0,2.0,1.0,1.0,0.0,1.0,7.0,6.0,4.0,1.0
1,2001-KW02,10.0,6.0,15.0,11.0,4.0,12.0,7.0,8.0,6.0,...,8.0,4.0,3.0,1.0,0.0,4.0,3.0,2.0,7.0,2.0
2,2001-KW03,18.0,3.0,12.0,12.0,6.0,10.0,13.0,14.0,6.0,...,16.0,4.0,2.0,1.0,1.0,4.0,5.0,2.0,4.0,1.0
3,2001-KW04,12.0,8.0,21.0,13.0,6.0,19.0,9.0,21.0,11.0,...,12.0,2.0,11.0,0.0,5.0,8.0,11.0,2.0,7.0,1.0
4,2001-KW05,11.0,13.0,16.0,2.0,8.0,14.0,13.0,22.0,6.0,...,7.0,9.0,6.0,2.0,2.0,11.0,15.0,3.0,7.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,2019-KW48,1.0,1.0,2.0,0.0,0.0,2.0,0.0,3.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,2019-KW49,2.0,1.0,2.0,0.0,0.0,1.0,1.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
974,2019-KW50,3.0,3.0,6.0,1.0,3.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
975,2019-KW51,6.0,7.0,20.0,0.0,2.0,1.0,2.0,2.0,2.0,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0


In [7]:
conn = sqlite3.connect("/home/lucas/Downloads/coronapolation.sqlite")
df = pd.read_sql_query("SELECT * FROM RKI_COVID19;", conn)
df.tail()



,ObjectId,Meldedatum,IdBundesland,Bundesland,IdLandkreis,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall
358645,639769,2020-03-31T00:00:00.000Z,16,Thüringen,16077,LK Altenburger Land,A60-A79,M,2,0
358646,639770,2020-03-16T00:00:00.000Z,16,Thüringen,16077,LK Altenburger Land,A60-A79,W,1,0
358647,639771,2020-03-27T00:00:00.000Z,16,Thüringen,16077,LK Altenburger Land,A60-A79,W,1,0
358648,639772,2020-03-27T00:00:00.000Z,16,Thüringen,16077,LK Altenburger Land,A80+,M,1,0
358649,639773,2020-03-24T00:00:00.000Z,16,Thüringen,16077,LK Altenburger Land,A80+,W,1,0


In [ ]:
for index, row in df.iterrows():
    print(row['date/Kreis'])

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

In [ ]:
TRAIN_SPLIT = 900

In [ ]:
tf.random.set_seed(11)

In [ ]:
uni_data = df['LK München']
uni_data.index = df['date/Kreis']
uni_data

In [ ]:
uni_data.plot(subplots=True)

In [ ]:
uni_data = uni_data.values

In [ ]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()
print(uni_train_mean)
print(uni_train_std)

In [ ]:
uni_data = (uni_data-uni_train_mean)/uni_train_std
print(uni_data)

In [ ]:
univariate_past_history = 13
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target infected peoble to predict to predict')
print (y_train_uni[0])

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')

In [ ]:
def baseline(history):
  return np.mean(history)

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
BATCH_SIZE = 25
BUFFER_SIZE = 1000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
print(train_univariate)

In [ ]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

In [ ]:
for x, y in val_univariate.take(1):
    print(simple_lstm_model.predict(x).shape)

In [ ]:
EVALUATION_INTERVAL = 900
EPOCHS = 10

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

In [ ]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
  plot.show()